In [1]:
import json, os, itertools
from pathlib import Path
from typing import List, Dict, Tuple

In [2]:
SPECIALIZATION = "AI & ML"

SPECIFIED_FIXED = ["SYDE 600", "SYDE 660A", "SYDE 675"]      # 必修
SPECIFIED_OR    = [("SYDE 522", "SYDE 552")]    # 二选一
ELECTIVE_LIST   = ["SYDE 662", "SYDE 671",
                   "SYDE 672", "SYDE 673", "SYDE 674"]

# 每学期容量 (MEng 典型 3-3-2)
SEM_CAPACITY = [3, 3, 2]


In [3]:
TERM_MAP = {1: "Winter", 5: "Spring", 9: "Fall"}

def next_term_code(code: str) -> str:
    """给定学期代码 '1251' → 下一个代码 '1255'."""
    year = int(code[:2])
    digit = int(code[2])
    next_digit = {1: 5, 5: 9, 9: 1}[digit]
    if next_digit == 1:
        year += 1
    return f"{year:02d}{next_digit}"

In [4]:
def term_sequence(start: str, n: int) -> List[str]:
    seq = [start]
    for _ in range(n - 1):
        seq.append(next_term_code(seq[-1]))
    return seq

In [5]:
def load_offerings(term_codes: List[str],
                   folder: Path = Path(".")) -> Dict[str, List[str]]:
    """
    返回 dict:  course_code -> [term_code, ...]
    若某学期文件缺失则忽略
    """
    offers = {}
    for tc in term_codes:
        file_path = folder / f"{tc}.json"
        if not file_path.exists():
            continue
        data = json.loads(file_path.read_text(encoding="utf-8"))
        for c in data:
            code = f"{c['subjectCode']} {c['catalogNumber']}"
            offers.setdefault(code, []).append(tc)
    return offers

In [6]:
def human_term(tc: str) -> str:
    return f"{TERM_MAP[int(tc[2])]} 20{tc[:2]}"

In [7]:
def main():
    # === 1. 用户输入 ===
    start_tc   = input("起始学期代码 (e.g. 1251): ").strip()
    num_terms  = int(input("规划学期数 (e.g. 3): ").strip())

    # === 2. 生成学期序列 ===
    terms = term_sequence(start_tc, num_terms)
    print("\n学期序列:", [human_term(t) for t in terms])

    # === 3. 读取本地 JSON / API 数据 ===
    offerings = load_offerings(terms, Path("./final_verified_course_data/SYDE"))

    # === 4. Specified 课程选择 (522 *或* 552) ===
    chosen_or = None
    for cand in SPECIFIED_OR[0]:          # 只有一组二选一
        if cand in offerings:
            chosen_or = cand
            break
    specified_courses = SPECIFIED_FIXED + ([chosen_or] if chosen_or else [])

    # === 5. 统计出现情况 ===
    rows: List[Tuple[str, str, str]] = []   # 课程, 类型, 出现学期
    def push(code, typ):
        terms_found = ", ".join(human_term(t) for t in offerings.get(code, []))
        rows.append((code, typ, terms_found or "❌ 未开"))

    for c in SPECIFIED_FIXED:
        push(c, "Specified")
    push("SYDE 522 / 552", "Specified-(1 of 2)")  # 汇总显示
    for c in ELECTIVE_LIST:
        push(c, "Elective")

    # === 6. 打印统计表 ===
    print("\n==== 课程出现统计 ====\n")
    print(f"{'Course':<20}{'Type':<18}{'Terms Offered'}")
    print("-"*60)
    for code, typ, termstr in rows:
        print(f"{code:<20}{typ:<18}{termstr}")

    # === 7. 打印空白课表模板 ===
    print("\n==== 空白课表 (容量 3-3-2) ====")
    for idx, tc in enumerate(terms):
        slots = ["[ ]"] * SEM_CAPACITY[idx]
        print(f"{human_term(tc):<12}", "  ".join(slots))

    # === 8. 简要建议 ===
    print("\n提示：")
    print(" 1) 先把所有 Specified 课程放入对应开课学期。")
    print(" 2) Elective 列表中至少选择 1 门; 如列表中无课可选, 需要延长规划。")
    print(" 3) 其余空位可填自由 SYDE / 非 SYDE 课程, 保证总数≥8, SYDE≥5.")


In [9]:
if __name__ == "__main__":
    main()

起始学期代码 (e.g. 1251):  1251
规划学期数 (e.g. 3):  3



学期序列: ['Spring 2012', 'Fall 2012', 'Winter 2013']

==== 课程出现统计 ====

Course              Type              Terms Offered
------------------------------------------------------------
SYDE 660A           Specified         ❌ 未开
SYDE 675            Specified         Spring 2012
SYDE 522 / 552      Specified-(1 of 2)❌ 未开
SYDE 662            Elective          ❌ 未开
SYDE 671            Elective          ❌ 未开
SYDE 672            Elective          ❌ 未开
SYDE 673            Elective          Spring 2012
SYDE 674            Elective          ❌ 未开

==== 空白课表 (容量 3-3-2) ====
Spring 2012  [ ]  [ ]  [ ]
Fall 2012    [ ]  [ ]  [ ]
Winter 2013  [ ]  [ ]

提示：
 1) 先把所有 Specified 课程放入对应开课学期。
 2) Elective 列表中至少选择 1 门; 如列表中无课可选, 需要延长规划。
 3) 其余空位可填自由 SYDE / 非 SYDE 课程, 保证总数≥8, SYDE≥5.
